In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import re

## Define methods for merging the data

In [2]:
def newdatabase():
    newdatabase = pd.DataFrame({'Description':[], 'Protein\nDB number':[],"# AA's in\nprotein":[]})
    newdatabase.set_index('Description',inplace=True)
    return newdatabase

### Open a mass spec excell file and getting the protein sheet

In [3]:
def getproteins(xlname):
    xl_file=pd.ExcelFile(xlname)
    proteins=xl_file.parse("Proteins")
    return proteins

### Split gene Description into different columns

In [4]:
def Split_Description(merged):
    
    merged.index = merged.index.astype(str)

    merged['Uniprot']=merged.index.str.extract(r'(\|.+\|)', expand=False).str.strip('|')
    merged['uniprotgene']=merged.index.str.extract("(\|[^/|\s]+\s)", expand=False).str.strip('|')
    merged['Protein']=merged.index.str.extract("(\s.+OS)", expand=False).str.strip()
    merged['Gene']=merged.index.str.extract("(GN=[\d|\w]+)", expand=False).str.replace('GN=','')
    merged['ugene']=merged.uniprotgene.str.replace('_HUMAN','')
    return merged

### Extract protein names from all the xclx files in the directory, remove gaps, and make one list

In [5]:
def make_name_list():
    merged=newdatabase()
    path = Path()
     
    for file in path.glob("*_Byonic.xlsx"):
        print(file)
        proteins = getproteins(file)
        proteins.set_index('Description', inplace=True)
        # Remove gaps
        nogaps=proteins.dropna(subset=["# of\nspectra"])

#         get the name of the condition
        start=file.name.find('UCSF')+10
        end=file.name.find('.raw')
        name_clean=file.name[start:end]
        
#         rename the spectre column
        nogaps.rename(columns={"# of\nspectra":name_clean},inplace=True)
    
#         Add values for each specific condition + for protein nDB and #Amino acids of the protein     
        merged=pd.merge(merged,nogaps[[name_clean]], on=["Description"], how='outer')
        merged.update(nogaps[['Protein\nDB number',"# AA's in\nprotein"]])


    merged= Split_Description(merged)
    return merged
    

### Combining the data together using the functions above

In [6]:
merged=make_name_list()

160920_SRamundo_Niv_Walter_UCSF_6023_WT_0.raw_20161005_Byonic.xlsx


C:\Users\nivdo\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


160920_SRamundo_Niv_Walter_UCSF_6023_WT_40.raw_20161005_Byonic.xlsx
160920_SRamundo_Niv_Walter_UCSF_6023_WT_80.raw_20161005_Byonic.xlsx


In [7]:
# Add 0s instead of NaN
merged=merged.fillna(0)

In [8]:
pd.set_option('display.max_columns', None)
merged.head()

,Protein\nDB number,# AA's in\nprotein,WT_0,WT_40,WT_80,Uniprot,uniprotgene,Protein,Gene,ugene
Description,,,,,,,,,,
">sp|P04264|K2C1_HUMAN Keratin, type II cytoskeletal 1 OS=Homo sapiens GN=KRT1 PE=1 SV=6",17682.0,644.0,486.0,244.0,119.0,P04264,K2C1_HUMAN,"Keratin, type II cytoskeletal 1 OS",KRT1,K2C1
">sp|P13645|K1C10_HUMAN Keratin, type I cytoskeletal 10 OS=Homo sapiens GN=KRT10 PE=1 SV=6",18270.0,584.0,251.0,253.0,172.0,P13645,K1C10_HUMAN,"Keratin, type I cytoskeletal 10 OS",KRT10,K1C10
>sp|K1C9_HUMAN|(Common contaminant protein),175619.0,623.0,227.0,0.0,0.0,K1C9_HUMAN,(Common,0,0,(Common
">sp|P35908|K22E_HUMAN Keratin, type II cytoskeletal 2 epidermal OS=Homo sapiens GN=KRT2 PE=1 SV=2",17681.0,639.0,119.0,106.0,80.0,P35908,K22E_HUMAN,"Keratin, type II cytoskeletal 2 epidermal OS",KRT2,K22E
>sp|KRHB4_HUMAN|(Common contaminant protein),175648.0,600.0,94.0,0.0,0.0,KRHB4_HUMAN,(Common,0,0,(Common


In [9]:
merged.shape

(141, 10)

### Save a copy of the database before we clean it

In [10]:
merged.to_csv('Data before filtering.csv')

In [11]:
merged.columns

Index(['Protein\nDB number', '# AA's in\nprotein', 'WT_0', 'WT_40', 'WT_80',
       'Uniprot', 'uniprotgene', 'Protein', 'Gene', 'ugene'],
      dtype='object')

## Combine the data of genes that their reads where splitted into different isoforms and different annotations

### Group by the UNIPROT GENE NAME - only the columns that contains the spectra counts

In [79]:
exp_names=merged.columns.to_list()[3:-5]

In [80]:
df_sum=merged.groupby(['uniprotgene'])[exp_names].sum()

In [81]:
df_sum.head()

,WT_0,WT_40,WT_80
uniprotgene,,,
(Common,481.0,116.0,106.0
A0A075B6N9_HUMAN,0.0,0.0,1.0
A0A0A0MSA7_HUMAN,0.0,0.0,8.0
A2KUC4_HUMAN,0.0,38.0,0.0
ABHEB_HUMAN,0.0,3.0,0.0


In [28]:
df_sum.shape

(132, 3)

In [45]:
merged=merged.reset_index()

In [46]:
merged.head()

,Description,Protein\nDB number,# AA's in\nprotein,WT_0,WT_40,WT_80,Uniprot,uniprotgene,Protein,Gene,ugene
0,">sp|P04264|K2C1_HUMAN Keratin, type II cytoske...",17682.0,644.0,486.0,244.0,119.0,P04264,K2C1_HUMAN,"Keratin, type II cytoskeletal 1 OS",KRT1,K2C1
1,">sp|P13645|K1C10_HUMAN Keratin, type I cytoske...",18270.0,584.0,251.0,253.0,172.0,P13645,K1C10_HUMAN,"Keratin, type I cytoskeletal 10 OS",KRT10,K1C10
2,>sp|K1C9_HUMAN|(Common contaminant protein),175619.0,623.0,227.0,0.0,0.0,K1C9_HUMAN,(Common,0,0,(Common
3,">sp|P35908|K22E_HUMAN Keratin, type II cytoske...",17681.0,639.0,119.0,106.0,80.0,P35908,K22E_HUMAN,"Keratin, type II cytoskeletal 2 epidermal OS",KRT2,K22E
4,>sp|KRHB4_HUMAN|(Common contaminant protein),175648.0,600.0,94.0,0.0,0.0,KRHB4_HUMAN,(Common,0,0,(Common


### Add the remaining descriptive columns

In [47]:
subset=merged[['Description','Protein\nDB number', "# AA's in\nprotein", 'Protein', 'Gene','Uniprot','uniprotgene']]
subset.columns

Index(['Description', 'Protein\nDB number', '# AA's in\nprotein', 'Protein',
       'Gene', 'Uniprot', 'uniprotgene'],
      dtype='object')

In [48]:
df_sum_all=pd.merge(df_sum, subset, how='left', on='uniprotgene' )

In [49]:
df_sum_all.head()

,uniprotgene,WT_0,WT_40,WT_80,Description,Protein\nDB number,# AA's in\nprotein,Protein,Gene,Uniprot
0,(Common,481.0,116.0,106.0,>sp|K1C9_HUMAN|(Common contaminant protein),175619.0,623.0,0,0,K1C9_HUMAN
1,(Common,481.0,116.0,106.0,>sp|KRHB4_HUMAN|(Common contaminant protein),175648.0,600.0,0,0,KRHB4_HUMAN
2,(Common,481.0,116.0,106.0,>sp|TRYP_PIG|(Common contaminant protein),175660.0,231.0,0,0,TRYP_PIG
3,(Common,481.0,116.0,106.0,>sp|CAS1_BOVIN|(Common contaminant protein),175678.0,214.0,0,0,CAS1_BOVIN
4,(Common,481.0,116.0,106.0,>sp|CASB_BOVIN|(Common contaminant protein),175680.0,224.0,0,0,CASB_BOVIN


In [50]:
df_sum_all.drop_duplicates(subset=['uniprotgene'], inplace=True) 
df_sum_all.reset_index(drop=True, inplace=True)

### Now grouping by Gene name (not uniprot)

In [51]:
df_sum_double=df_sum_all.groupby(['Gene'])[exp_names].sum()

### Bring back the info columns we dropped before the grouping

In [52]:
df_sum_double=pd.merge(df_sum_double, subset, how='left', on='Gene' )

In [53]:
df_sum_double.head()

,Gene,WT_0,WT_40,WT_80,Description,Protein\nDB number,# AA's in\nprotein,Protein,Uniprot,uniprotgene
0,0,500.0,155.0,121.0,>sp|K1C9_HUMAN|(Common contaminant protein),175619.0,623.0,0,K1C9_HUMAN,(Common
1,0,500.0,155.0,121.0,>sp|KRHB4_HUMAN|(Common contaminant protein),175648.0,600.0,0,KRHB4_HUMAN,(Common
2,0,500.0,155.0,121.0,>sp|TRYP_PIG|(Common contaminant protein),175660.0,231.0,0,TRYP_PIG,(Common
3,0,500.0,155.0,121.0,">tr|B4DE59|B4DE59_HUMAN cDNA FLJ60424, highly ...",59366.0,563.0,"cDNA FLJ60424, highly similar to Junction plak...",B4DE59,B4DE59_HUMAN
4,0,500.0,155.0,121.0,>sp|CAS1_BOVIN|(Common contaminant protein),175678.0,214.0,0,CAS1_BOVIN,(Common


Before mannualy annotation the size was 1546

In [54]:
df_sum_double.shape

(141, 10)

In [55]:
df_sum_double.drop_duplicates(subset=['Gene'], inplace=True) 
df_sum_double.reset_index(drop=True, inplace=True)

In [56]:
df_sum_double.shape

(119, 10)

1657-1442 = 215 gene isoforms were condensed

In [57]:
df_sum_double.head()

,Gene,WT_0,WT_40,WT_80,Description,Protein\nDB number,# AA's in\nprotein,Protein,Uniprot,uniprotgene
0,0,500.0,155.0,121.0,>sp|K1C9_HUMAN|(Common contaminant protein),175619.0,623.0,0,K1C9_HUMAN,(Common
1,ABHD14B,0.0,3.0,0.0,>sp|Q96IU4|ABHEB_HUMAN Protein ABHD14B OS=Homo...,465.0,210.0,Protein ABHD14B OS,Q96IU4,ABHEB_HUMAN
2,ACACA,7.0,1.0,9.0,>sp|Q13085|ACACA_HUMAN Acetyl-CoA carboxylase ...,562.0,2346.0,Acetyl-CoA carboxylase 1 OS,Q13085,ACACA_HUMAN
3,ACADVL,5.0,0.0,0.0,>Reverse >sp|P49748-3|ACADV_HUMAN Isoform 3 of...,176160.0,678.0,>sp|P49748-3|ACADV_HUMAN Isoform 3 of Very lon...,P49748-3,ACADV_HUMAN
4,ACSBG1,3.0,0.0,5.0,>Reverse >sp|Q96GR2|ACBG1_HUMAN Long-chain-fat...,176162.0,724.0,>sp|Q96GR2|ACBG1_HUMAN Long-chain-fatty-acid--...,Q96GR2,ACBG1_HUMAN


### Save the data with genes isoforms combined together

In [58]:
df_sum_double.to_csv('Combined heatmap dat after grouping before dropping contaminants.csv')

### Getting rid of all rows of the reverse dataset - junk

In [59]:
non_reverse=~df_sum_double['Description'].str.contains('Reverse')
df_sum_double= df_sum_double[non_reverse]
df_sum_double.shape

(88, 10)

### Delete Keratin genes (contaminants)

In [60]:
non_Keratin=~df_sum_double['Description'].str.contains('Keratin')
df_sum_double= df_sum_double[non_Keratin]
df_sum_double.shape

(69, 10)

In [61]:
df_sum_double.to_csv('Combined heatmap eIF2B clean.csv')

## Annotate the genes using Uniprot

In [62]:
from bioservices.uniprot import UniProt
from bs4 import BeautifulSoup, Comment

In [63]:
df_sum_double.set_index(['Uniprot'], inplace=True)

### Get gene names

In [64]:
uniprotids= df_sum_double.index.to_list()
uniprotids[0:10]

['K1C9_HUMAN',
 'Q96IU4',
 'Q13085',
 'P60709',
 'Q562R1',
 'Q12802-2',
 'Q5JQC9',
 'P07355',
 'P05089',
 'Q13017']

In [65]:
u = UniProt(verbose=False)

In [66]:
df_sum_double['Locations']=''
df_sum_double['Info']=''
df_sum_double.head()

,Gene,WT_0,WT_40,WT_80,Description,Protein\nDB number,# AA's in\nprotein,Protein,uniprotgene,Locations,Info
Uniprot,,,,,,,,,,,
K1C9_HUMAN,0,500.0,155.0,121.0,>sp|K1C9_HUMAN|(Common contaminant protein),175619.0,623.0,0,(Common,,
Q96IU4,ABHD14B,0.0,3.0,0.0,>sp|Q96IU4|ABHEB_HUMAN Protein ABHD14B OS=Homo...,465.0,210.0,Protein ABHD14B OS,ABHEB_HUMAN,,
Q13085,ACACA,7.0,1.0,9.0,>sp|Q13085|ACACA_HUMAN Acetyl-CoA carboxylase ...,562.0,2346.0,Acetyl-CoA carboxylase 1 OS,ACACA_HUMAN,,
P60709,ACTB,9.0,1.0,0.0,">sp|P60709|ACTB_HUMAN Actin, cytoplasmic 1 OS=...",607.0,375.0,"Actin, cytoplasmic 1 OS",ACTB_HUMAN,,
Q562R1,ACTBL2,0.0,0.0,2.0,>sp|Q562R1|ACTBL_HUMAN Beta-actin-like protein...,1249.0,376.0,Beta-actin-like protein 2 OS,ACTBL_HUMAN,,


### Add information + subcellular localization for each protein from Uniprot API

In [67]:
count=0
for uniprotid in uniprotids:
    count+= 1
    print(count, uniprotid)
    info_raw=u.search(uniprotid, frmt="xml")
    soup = BeautifulSoup(info_raw,'xml')
    locs= soup.find_all('subcellularLocation')
    loc_list=[]
    evedents_list=[]
    for loc in locs:
        loc_list.append(str.strip(loc.get_text()))
#         evedents_list.append(loc.location.attrs['evidence'])
    df_sum_double.at[uniprotid,'Locations']= loc_list
    info=soup.find('comment')
    if info:
        df_sum_double.at[uniprotid,'Info']= str.strip(info.text)

1 K1C9_HUMAN
2 Q96IU4
3 Q13085
4 P60709
5 Q562R1
6 Q12802-2
7 Q5JQC9
8 P07355
9 P05089
10 Q13017
11 P25311
12 Q8NFC6
13 P0C0L4
14 Q9NZT1
15 P31944
16 Q9UGN4
17 Q15517
18 Q2VPA4
19 L0R5A1
20 Q8TB45
21 A0PJE2-3
22 Q8IY37
23 Q6E0U4-2
24 Q08554
25 Q02413
26 P15924
27 Q4G0W2
28 L8E9W3
29 O95864
30 Q9NUU6
31 Q2V2M9
32 P20930
33 Q5D862
34 Q8IVA8
35 Q3KR37
36 Q86XA9
37 Q16778
38 Q86YZ3
39 Q4G0P3
40 A0A075B6N9
41 Q14643
42 Q7L273
43 Q96Q89
44 Q6ZMV9
45 P50748
46 P47929
47 P61626
48 Q96RQ3
49 Q02509
50 P11498
51 Q694B2
52 Q2PPJ7
53 B9EJE3
54 P62241
55 P31040
56 Q9UHI5
57 Q9BX66-12
58 Q13813
59 B4DGK4
60 Q96C24
61 O95759
62 Q8NDV7
63 Q71U36
64 A6NHL2
65 P10599
66 P62987
67 Q70CQ4
68 P08670
69 Q9H6S0


In [68]:
df_sum_double.head()

,Gene,WT_0,WT_40,WT_80,Description,Protein\nDB number,# AA's in\nprotein,Protein,uniprotgene,Locations,Info
Uniprot,,,,,,,,,,,
K1C9_HUMAN,0,500.0,155.0,121.0,>sp|K1C9_HUMAN|(Common contaminant protein),175619.0,623.0,0,(Common,[],May serve an important special function either...
Q96IU4,ABHD14B,0.0,3.0,0.0,>sp|Q96IU4|ABHEB_HUMAN Protein ABHD14B OS=Homo...,465.0,210.0,Protein ABHD14B OS,ABHEB_HUMAN,"[Cytoplasm, Nucleus]",Has hydrolase activity towards p-nitrophenyl b...
Q13085,ACACA,7.0,1.0,9.0,>sp|Q13085|ACACA_HUMAN Acetyl-CoA carboxylase ...,562.0,2346.0,Acetyl-CoA carboxylase 1 OS,ACACA_HUMAN,[Cytoplasm\nCytosol],Cytosolic enzyme that catalyzes the carboxylat...
P60709,ACTB,9.0,1.0,0.0,">sp|P60709|ACTB_HUMAN Actin, cytoplasmic 1 OS=...",607.0,375.0,"Actin, cytoplasmic 1 OS",ACTB_HUMAN,"[Cytoplasm\nCytoskeleton, Nucleus]",Actin is a highly conserved protein that polym...
Q562R1,ACTBL2,0.0,0.0,2.0,>sp|Q562R1|ACTBL_HUMAN Beta-actin-like protein...,1249.0,376.0,Beta-actin-like protein 2 OS,ACTBL_HUMAN,[Cytoplasm\nCytoskeleton],Actins are highly conserved proteins that are ...


### Save the annotated data

In [70]:
df_sum_double.to_csv('Combined heatmap  plus localization and description.csv')